In [1]:
%matplotlib inline
import sympy as sp
import numpy as np
import pprint

from mueller_matrices import M_Retarder, M_Diattenuator, M_rotate

## Define symbolic parameters

In [2]:
S = sp.Matrix(sp.symbols('s0:4'))
s0, s1, s2, s3 = sp.symbols('s0, s1, s2, s3')
x, eps, eps2, theta = sp.symbols('x, eps, eps2, theta')
Tmax, Tmin = sp.symbols('Tmax, Tmin')
Tmax_0, Tmin_0 = sp.symbols('Tmax_0, Tmin_0')
Tmax_90, Tmin_90 = sp.symbols('Tmax_90, Tmin_90')
tx, ty, rx, ry = sp.symbols('tx, ty, rx, ry')
t90, t45, t135 = sp.symbols('t90, t45, t135')
delta = sp.symbols('delta')

## Model for 0 and 90 deg polarizers

In [3]:
# Max and min transmission of the two polarizers
Tmax_0 = 0.9718; Tmin_0 = 6.4E-4;
Tmax_90 = 0.9681; Tmin_90 = 6.3E-4;

M0_polarizer = M_Diattenuator(0, Tmax_0, Tmin_0)
M90_polarizer = M_Diattenuator(sp.pi/2, Tmax_90, Tmin_90)

M0_polarizer_ideal = M_Diattenuator(0, 1, 0)
M90_polarizer_ideal = M_Diattenuator(sp.pi/2, 1, 0)

## Model for 50/50 splitter

Splitter is modelled as a linear diattenuator, which transmits or reflects 50% of the incident light. Non-ideal performance is given by unequal transmission/reflection of s- and p-polarized light

In [4]:
# Transmission and reflection coefficients along two orthogonal coordinates
tx = 0.5321; ty = 0.4430;
rx = 0.4080; ry = 0.5091;

M5050_transmission = M_Diattenuator(0, tx, ty)
M5050_reflection = M_Diattenuator(0, rx, ry)

M5050_ideal = M_Diattenuator(0, 0.5, 0.5)

## Model for half-waveplate in 45/135 paths

In [5]:
# Orientation and retardance of the HWP
# HWP is oriented at -22.5 deg in current implementation
hwp_theta = -0.3592; hwp_delta = 3.0604;

M_HWP = M_Retarder(hwp_theta, hwp_delta)
M_HWP_ideal = M_Retarder(-sp.pi/8, sp.pi)

## Transmission coefficients of the 45/90/135 paths

In [6]:
t45 = 1.0056; t90 = 0.8720; t135 = 0.8174;

## Mueller matrices for the four paths

In [14]:
M0 = M0_polarizer * M5050_transmission
M90 = t90 * M90_polarizer * M5050_transmission
M45 = t45 * M_Diattenuator(0, 1, 0) * M_HWP * M5050_reflection # Linear polarizer is modeled as ideal
M135 = t135 * M_Diattenuator(sp.pi/2, 1, 0) * M_HWP * M5050_reflection # Linear polarizer is modeled as ideal

M0_ideal = M0_polarizer_ideal * M5050_ideal
M90_ideal = M90_polarizer_ideal * M5050_ideal
M45_ideal = M_Diattenuator(0, 1, 0) * M_HWP_ideal * M5050_ideal
M135_ideal = M_Diattenuator(sp.pi/2, 1, 0) * M_HWP_ideal * M5050_ideal

## Instrument matrix and inverse instrument matrix

* I = A * S

* S = Ainv * I

In [22]:
A = np.matrix([M0[0,:], M45[0,:], M90[0,:], M135[0,:],], dtype='float')
A_ideal = np.matrix([M0_ideal[0,:], M45_ideal[0,:], M90_ideal[0,:], M135_ideal[0,:],], dtype='float')

A_inv = np.linalg.pinv(A)
A_inv_ideal = np.linalg.pinv(A_ideal)

A_fit = np.matrix([[0.2642, 0.2600, 0.0011, 0],
                   [0.2282, 0.0058, -0.2258, 0],
                   [0.1845, -0.1876, -0.0005, 0],
                   [0.1909, -0.0460, 0.1846, 0]])
A_fit_inv = np.linalg.pinv(A_fit)

In [23]:
pp = pprint.PrettyPrinter()
pp.pprint(A_fit)
pp.pprint(A_fit_inv)

matrix([[ 0.2642,  0.26  ,  0.0011,  0.    ],
        [ 0.2282,  0.0058, -0.2258,  0.    ],
        [ 0.1845, -0.1876, -0.0005,  0.    ],
        [ 0.1909, -0.046 ,  0.1846,  0.    ]])
matrix([[ 1.10274352,  1.03657126,  1.25025407,  1.26473402],
        [ 2.23994121, -0.43269806, -2.10502273, -0.5483189 ],
        [ 0.46959284, -2.506331  , -0.00255037,  2.34860581],
        [ 0.        ,  0.        ,  0.        ,  0.        ]])
